<a href="https://colab.research.google.com/github/Hugues7/gith_init/blob/main/TP1_StickGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="./assets/Logo_ESEO_GROUPE.jpg" alt="Tech Logo" align="center" height="400" width="400"/>

<h1 align="center"; style="color:#3333cc;font-size:55px">Apprentissage par renforcement</h1>

<h2 align="center"; style="color:#0099cc;font-size:30px">TP1 : Développez un agent d'apprentissage par renforcement pour jouer au StickGame</h2>

Pour ce TP, vous implementerez un environnement de jeu de Stick Game ainsi qu'un agent capable d'exceller dans cet environnement. Utilisez ce notebook Jupyter qui passe en revue chaque partie du TP pour présenter des preuves et une analyse de vos résultats.

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>📥 <strong>Import des librairies </strong></span></h2>

In [64]:
from random import randint
import random
import numpy as np

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>💾 <strong>Les règles du jeu</strong></span></h2>

<img src="./assets/stickgame.jpg" alt="Tech Logo" align="center" height="400" width="400"/>

Le jeu se joue à deux joueurs. Le joueur qui commence est tiré au sort. Le jeu commence avec un nombre prédéfini de batonnets (souvent 12 ou 16). Dans ce TP, nous commencerons le jeu avec 12 batonnets. Chaque joueur enlève au choix 1, 2 ou 3 bâtonnets puis c'est au tour de l'adversaire de retirer 1, 2 ou 3 bâtonnets. Celui qui enlèvera le dernier bâtonnet sera le perdant.

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🧠<strong>1. Formalisation du problème d'apprentissage par renforcement</strong></span></h2>

Un problème d'apprentissage par renforcement nécessite forcément des interactions entre un agent et un environnement.


<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quels sont les principales données que l'environnement doit fournir à l'agent ?</span></p>






L'environnement doit fournir à l'agent au moins les informations suivantes :

* **L'état actuel de l'environnement:**  correspondant au nombre de bâtonnets restants.
* **La récompense (ou pénalité):**
* **Les actions possibles**  
* **Un indicateur de fin de partie (état terminal):**

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  En fonction de la règle de jeu du Stick Game énoncée précédemment, quelles valeurs peut-on attribuer à l'état et au récompenses renvoyées par l'environnement ? Quel est l'état initial ?</span></p>

In [65]:
INITIAL_STATE =12
FAIL_REWARD   = -1
GAME_IN_PROGRESS_REWARD = 0
WIN_REWARD = 1

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quelle est la principale donnée que l'agent fournit à l'environnement ?</span></p>

- ***le nombre de bâtonnets qu'il souhaite retirer (1, 2 ou 3)***.

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  En fonction de la règle de jeu du Stick Game énoncée précédemment, quelles sont les actions possibles dans l'espace des actions ? (Exprimez-les sous forme de liste Python)</span></p>

In [66]:
ACTION_SPACE = [1, 2, 3]

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🌎 <strong>2. Conception de l'environnement</strong></span></h2>

L'environnement sera représenté sous forme d'une classe Python ayant des méthodes qui lui permettent d'interagir avec l'agent et l'utilisateur du notebook (c'est-à-dire vous).
Référez-vous directement aux docstrings de la classe pour comprendre l'utilité de chaque attribut et méthode de la classe

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de développer les méthodes de classe <strong>is_finished()</strong>, <strong>reset()</strong>, <strong>display()</strong>, <strong>step()</strong></span></p>

In [67]:
class StickGame(object):
    '''
    Classe StickGame représentant l'environnement du jeu des batonnets

    '''

    def __init__(self, nb):
        # Comme dans toutes les classes, la fonction __init__ est la fonction d'initialisation.
        # Cette fonction est appelée à chaque fois qu'un objet de cette classe est instanciée
        # Au moment de l'initialisation, les attributs de l'objet sont définis

        super(StickGame, self).__init__()
        # L'attribut original_nb est le nombre de batonnets au début de chaque partie.
        # Cet attribut n'est pas modifiée au cours du jeu et permet juste de ré-initialiser le jeu en fin de partie
        self.original_nb = nb

        # L'attribut nb est le nombre de batonnets à tout moment au cours de la partie.
        # Cet attribut est mis à jour à chaque fois qu'un joueur joue
        self.nb = nb

    def is_finished(self):

        '''
        Méthode permettant de vérifier si le jeu est terminé ou non.
        /!\ : Attention ! Prenez en compte le cas d'un nombre de batonnets négatif à la fin du jeu

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Booléen : True si le jeu est terminé, False sinon
        '''
        if self.nb <= 0:
            return True
        else:
            return False


    def reset(self):

        '''
        Méthode permettant de ré-initialiser le jeu à la fin de chaque partie

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        self.nb : Le nombre de batonnets au début de chaque partie selon la règle du jeu
        '''
        if self.nb > 0:
            self.nb = self.original_nb
            return self.nb
        else:
            return self.nb


    def display(self):

        '''
        Méthode permettant d'afficher le nombre de batonnets disponibles dans le jeu
        après chaque action de jeu des joueurs

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Rien : affiche avec print le nombre de batonnets restant disponibles dans le jeu
        '''
        if self.nb > 0:
          print ("| " * self.nb)

    def step(self, action):

        '''
        Méthode permettant de renvoyer à l'agent après chaque de sa part le nouvel état de l'environnement
        ainsi que la récompense obtenue. La méthode renvoie un nouvel état égal None si le jeu est terminé.
        /!\ : Attention ! Prenez en compte le cas d'un nombre de batonnets négatif à la fin du jeu


                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - action (int) : Nombre entier désignant l'action entreprise par l'agent


                Returns:
                        - current_state (int/None) : Nombre entier ou None (dans le cas où le jeu est terminé)
                                                     désignant l'état actuel du jeu
                        - reward (int) : Nombre entier désignant la récompense à fournir à l'agent
                                         en fonction de l'état du jeu

        '''
        if self.nb > 0:
            self.nb -= action
            if self.nb == 0:
              current_state = None
              reward = WIN_REWARD
            else:
                current_state = self.nb
                reward = GAME_IN_PROGRESS_REWARD


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  A présent, vous pouvez tester votre environnement </span></p>

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Instanciez une partie avec 12 batonnets initiaux </span></p>

In [68]:
game = StickGame(12)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [69]:
game.display()

| | | | | | | | | | | | 


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Faites l'action de retirer 2 batonnets  </span></p>

In [70]:
game.step(2)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [71]:
game.display()

| | | | | | | | | | 


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Faites l'action de retirer 3 batonnets  </span></p>

In [72]:
game.step(3)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [74]:
game.display()

| | | | | | | 


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ré-initialisez le jeu </span></p>

In [75]:
game.reset()

12

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Affichez le jeu </span></p>

In [76]:
game.display()

| | | | | | | | | | | | 


<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🤖 <strong>3. Conception de l'agent</strong></span></h2>

L'agent est l'entité clé d'un problème d'apprentissage par renforcement. C'est lui qui décide des actions à entreprendre en fonction de l'état de l'environnement.

L'agent sera représenté sous forme d'une classe Python ayant des méthodes qui lui permettent d'interagir avec l'agent et l'utilisateur du notebook.

Référez-vous directement aux docstrings de la classe pour comprendre l'utilité de chaque attribut et méthode de la classe

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de développer les méthodes de classe <strong>reset_stat()</strong>, <strong>greedy_step()</strong>, <strong>play()</strong>, <strong>add_transition()</strong> et <strong>train()</strong> </span></p>

In [77]:
class StickPlayer(object):
    '''
    Classe StickPlayer représentant un agent jouant au jeu des batonnets

    '''

    def __init__(self, is_human, size, name, trainable=True):
        # Comme dans toutes les classes, la fonction __init__ est la fonction d'initialisation.
        # Cette fonction est appelée à chaque fois qu'un objet de cette classe est instanciée
        # Au moment de l'initialisation, les attributs de l'objet sont définis

        super(StickPlayer, self).__init__()

        # L'attribut name est une chaine de caractère qui représente le nom du joueur
        self.name = name

        # L'attribut is_human est un booléen qui définit si l'agent à instancier est un agent humain
        # ou un agent artificiel (IA)
        self.is_human = is_human

        # L'attribut history est une liste Python dans laquelle seront enregistrées les trajectoires du jeu
        # au cours des différentes parties jouées (épisodes)
        # Une trajectoire est un ensemble des transitions observées au cours d'une partie (épisode)
        # Une transition est un tuple (S, A, R, S') où
        #  - S  : est un l'état courant de l'environnement
        #  - A  : est l'action entreprise par l'agent
        #  - R  : est la récompense donnée par l'environnement suite à l'action A de l'agent
        #  - S' : est le nouvel état de l'environnement après que l'agent ait réalisé l'action A
        self.history = []

        # L'attribut V est un dictionnaire Python qui représente la fonction de valeur de l'agent.
        # Chaque clé du dictionnaire représente chacun des états de l'environnement
        # La valeur associée à chaque clé est la valeur de l'état, c'est-à-dire à quel point il est bon
        # pour l'agent d'être dans ce état
        self.V = {}

        # On initialise le dictionnaire avec toutes les valeurs d'état à 0.
        for s in range(1, size+1):
            self.V[s] = 0.

        # L'attribut win_nb est un nombre entier représentant le nombre de victoires cumulées de l'agent
        # sur un ensemble de parties jouées.
        # Cet attribut sera utilisé à des fins de statistiques pour évaluer l'agent au terme de son entrainement
        self.win_nb = 0.

        # L'attribut lose_nb est un nombre entier représentant le nombre de défaites cumulées de l'agent
        # sur un ensemble de parties jouées
        # Cet attribut sera utilisé à des fins de statistiques pour évaluer l'agent au terme de son entrainement
        self.lose_nb = 0.

        # L'attribut rewards est une liste Python dans laquelle seront l'ensemble des récompenses obtenues
        # tout au long des parties jouées
        # Cet attribut sera utilisé à des fins de statistiques pour évaluer l'agent au terme de son entrainement
        self.rewards = []

        # L'attribut eps (epsilon) est un nombre réel représentant le coefficient d'exploration initial de l'agent,
        # c'est-à-dire la tendance de l'agent à entreprendre des actions aléatoires au lieu d'entreprendre
        # des actions permettant d'atteindre l'état avec la plus grande valeur
        # Cette valeur diminuera au fur et à mesure au cours de l'entrainement
        self.eps = 0.99

        # L'attribut trainable est un booléen représentant si l'agent instancié peut être entrainé ou non.
        # Par exemple, si l'agent est un joueur humain, aucun entrainement n'est nécessaire.
        self.trainable = trainable

    def reset_stat(self):

        '''
        Méthode permettant de ré-initialiser les statistiques de victoires et défaites de l'agent

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        Rien : ré-initialise à 0 les attributs win_nb, lose_nb, rewards.
        '''
        if self.trainable:
            self.win_nb = 0.
            self.lose_nb = 0.
            self.rewards = []
            self.eps = 0.99
            return None

    def greedy_step(self, state):

        '''
        Méthode permettant de déterminer l'action gloutonne (greedy) à entreprendre en fonction de l'état de l'environnement
        Une action gloutonne est une action d'exploitation basée sur la connaissance à priori de l'agent
        C'est-à-dire atteindre les états avec le plus de valeur.

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - state (int) : Nombre entier représentant l'état courant de l'environnement


                Returns:
                        - action (int) : Nombre entier représentant la meilleure action à entreprendre
        '''

        if self.V[state] == 0:
            return random.choice(ACTION_SPACE)
        else:
            actions = []
            for action in ACTION_SPACE:
                if state - action > 0:
                    actions.append((action, self.V[state - action]))
                elif state - action == 0:
                    actions.append((action, WIN_REWARD))
                else:
                     actions.append((action, FAIL_REWARD))
            return max(actions, key=lambda item:item[1])[0]



    def play(self, state):

        '''
        Méthode qui détermine :
         - Pour l'agent artificiel : l'action que l'agent doit entreprendre et transmettre à
           l'environnement(action gloutonne ou exploratrice)
         - Pour l'agent humain : l'action directement rentrée au clavier

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - state (int) : Nombre entier représentant l'état courant de l'environnement


                Returns:
                        - action (int) : Nombre entier représentant la meilleure action à entreprendre
        '''
        if self.is_human:
            while True:
                try:
                    action = int(input("Entrez votre action (1, 2 ou 3): "))
                    if action in ACTION_SPACE:
                        return action
                    else:
                        print("Action invalide. Veuillez entrer 1, 2 ou 3.")
                except ValueError:
                    print("Entrée invalide. Veuillez entrer un nombre entier.")
        else:
            if random.random() < self.eps:
                # Exploration
                action = random.choice(ACTION_SPACE)
                self.eps -= 0.01
            else:
                # Exploitation
                action = self.greedy_step(state)
            return action



    def add_transition(self, n_tuple):

        '''
        Méthode qui rajoute une transition à (S, A, R, S') à l'historique  des transitions et
        qui rajoute la récompense obtenue à la liste des récompenses

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe
                        - n_tuple (tuple) : Tuple de 4 éléments (S, A, R, S')


                Returns:
                        - Rien : transition rajoutée à l'historique et récompense rajoutée à la liste des récompenses
        '''
        if self.trainable:
            self.history.append(n_tuple)
            self.rewards.append(n_tuple[2])
            if n_tuple[2] == WIN_REWARD:
                self.win_nb += 1
            elif n_tuple[2] == FAIL_REWARD:
                self.lose_nb += 1


    def train(self):

        '''
        Méthode qui réalise l'entrainement de l'agent, c'est-à-dire qui ajuste par itération successives
        la fonction des valeurs pour chaque état de l'environnement à partir des transitions enregistrées.

                Parameters:
                        - self : l'instance de classe, permettant d'accéder à tous les attributs de la classe


                Returns:
                        - Rien : fonction de valeur mise à jour itérativement, puis historique vidé à la fin des itérations
        '''
        if self.trainable:
            for n_tuple in self.history:
                self.V[n_tuple[0]] = self.V[n_tuple[0]] + 0.


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  A présent, vous pouvez tester votre agent </span></p>

<div class="alert alert-danger">
  <strong>TEST DE L'AGENT</strong><br><br> Le but de cette partie est de tester le bon fonctionnement de l'agent. SI vous constatez des résultats incohérents, revenez en arière et modifiez le code de l'agent.
</div>

In [78]:
# Ceci est un exemple de fonction de valeur.
# Cette fonction de valeur devrait permettre à votre agent d'être pertinent
TEST_FUNCTION_VALUE = {1: -0.99,
                       2: 0.24,
                       3: 0.90,
                       4: 0.41,
                       5: -0.85,
                       6: 0.06,
                       7: 0.69,
                       8: 0.08,
                       9: -0.64,
                       10: 0.02,
                       11: 0.03,
                       12: 0.47}

# Un valeur de test de la récompense immédiate
reward = 0

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Instanciez un agent de jeu non humain entrainable nommé <code style="text-align: left; font-size: 16px; color:#0f767a">ai_player</code> </span></p>

In [79]:
ai_player = StickPlayer(is_human=False, size=12, name="ai_player", trainable=True)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Assignez  <code style="text-align: left; font-size: 16px; color:#0f767a">TEST_FUNCTION_VALUE</code> comme fonction de valeur de l'agent <code style="text-align: left; font-size: 16px; color:#0f767a">ai_player</code></span></p>

In [80]:
ai_player.V = TEST_FUNCTION_VALUE

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Réglez le taux d'exploration de l'agent à 0</span></p>

In [81]:
ai_player.eps = 0

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 4, appelez la méthode <code style="text-align: left; font-size: 16px; color:#0f767a">play</code> pour faire jouer l'agent <code style="text-align: left; font-size: 16px; color:#0f767a">ai_player</code>. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. L'action choisie est-elle pertinente pour gagner le jeu ?</span></p>

In [82]:
state = 4
# Get the action chosen by the AI player
action = ai_player.play(state)

# Calculate the value of the new game state
new_state = state - action

# Check if the chosen action is relevant to win the game
if new_state == 0:
    print(f"Action: {action}, New state: {new_state}, Result: Win")
elif new_state in ai_player.V:
    print(f"Action: {action}, New state: {new_state}, Value of new state: {ai_player.V[new_state]}")
    if ai_player.V[new_state] > 0:
      print("The action is likely to be good.")
    else:
      print("The action is likely not to be good.")
else:
  print(f"Action: {action}, New state: {new_state}, The new state is outside the defined value function.")



Action: 1, New state: 3, Value of new state: 0.9
The action is likely to be good.


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ajoutez la transition obtenue précédemment (rappel : la valeur de reward a été définie plus haut) dans l'historique de l'agent, puis affichez cet historique</span></p>

In [83]:
# Add the transition to the agent's history
ai_player.add_transition((state, action, reward, new_state))

# Display the agent's history
ai_player.history

[(4, 1, 0, 3)]

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 3, appelez la méthode <code style="text-align: left; font-size: 16px; color:#0f767a">play</code> pour faire jouer l'agent <code style="text-align: left; font-size: 16px; color:#0f767a">ai_player</code>. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. L'action choisie est-elle pertinente pour gagner le jeu ?</span></p>

In [84]:
state = 3

# Get the action chosen by the AI player
action = ai_player.play(state)

# Calculate the value of the new game state
new_state = state - action

# Check if the chosen action is relevant to win the game
if new_state == 0:
    print(f"Action: {action}, New state: {new_state}, Result: Win")
elif new_state in ai_player.V:
    print(f"Action: {action}, New state: {new_state}, Value of new state: {ai_player.V[new_state]}")
    if ai_player.V[new_state] > 0:
      print("The action is likely to be good.")
    else:
      print("The action is likely not to be good.")
else:
  print(f"Action: {action}, New state: {new_state}, The new state is outside the defined value function.")

Action: 3, New state: 0, Result: Win


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ajoutez la transition obtenue précédemment (rappel : la valeur de reward a été définie plus haut) dans l'historique de l'agent, puis affichez cet historique</span></p>

In [85]:
# Add the transition to the agent's history
ai_player.add_transition((state, action, reward, new_state))

# Display the agent's history
ai_player.history

[(4, 1, 0, 3), (3, 3, 0, 0)]

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Réglez le taux d'exploration de l'agent à 0.99</span></p>

In [86]:
state = 4
ai_player.eps = 0.99


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 4, appelez la méthode <code style="text-align: left; font-size: 16px; color:#0f767a">play</code> pour faire jouer l'agent <code style="text-align: left; font-size: 16px; color:#0f767a">ai_player</code>. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. L'action choisie est-elle pertinente pour gagner le jeu ?</span></p>

In [87]:
# Get the action chosen by the AI player
action = ai_player.play(state)

# Calculate the value of the new game state
new_state = state - action

# Check if the chosen action is relevant to win the game
if new_state == 0:
    print(f"Action: {action}, New state: {new_state}, Result: Win")
elif new_state in ai_player.V:
    print(f"Action: {action}, New state: {new_state}, Value of new state: {ai_player.V[new_state]}")
    if ai_player.V[new_state] > 0:
      print("The action is likely to be good.")
    else:
      print("The action is likely not to be good.")
else:
  print(f"Action: {action}, New state: {new_state}, The new state is outside the defined value function.")



Action: 3, New state: 1, Value of new state: -0.99
The action is likely not to be good.


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ajoutez la transition obtenue précédemment (rappel : la valeur de reward a été définie plus haut) dans l'historique de l'agent, puis affichez cet historique</span></p>

In [88]:
# Add the transition to the agent's history
ai_player.add_transition((state, action, reward, new_state))

# Display the agent's history
ai_player.history

[(4, 1, 0, 3), (3, 3, 0, 0), (4, 3, 0, 1)]

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 3, appelez la méthode <code style="text-align: left; font-size: 16px; color:#0f767a">play</code> pour faire jouer l'agent <code>ai_player</code>. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. L'action choisie est-elle pertinente pour gagner le jeu ?</span></p>

In [89]:
state = 4
# Get the action chosen by the AI player
action = ai_player.play(state)

# Calculate the value of the new game state
new_state = state - action

# Check if the chosen action is relevant to win the game
if new_state == 0:
    print(f"Action: {action}, New state: {new_state}, Result: Win")
elif new_state in ai_player.V:
    print(f"Action: {action}, New state: {new_state}, Value of new state: {ai_player.V[new_state]}")
    if ai_player.V[new_state] > 0:
      print("The action is likely to be good.")
    else:
      print("The action is likely not to be good.")
else:
  print(f"Action: {action}, New state: {new_state}, The new state is outside the defined value function.")


Action: 1, New state: 3, Value of new state: 0.9
The action is likely to be good.


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ajoutez la transition obtenue précédemment (Utilisez la valeur de reward = -1) dans l'historique de l'agent, puis affichez cet historique</span></p>

In [90]:
# Add the transition to the agent's history
ai_player.add_transition((state, action, reward, new_state))

# Display the agent's history
ai_player.history

[(4, 1, 0, 3), (3, 3, 0, 0), (4, 3, 0, 1), (4, 1, 0, 3)]

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Appelez la méthode <code>train</code> pour entrainer l'agent. La fonction de valeur a-t-elle changé ?</span></p>

In [91]:
ai_player.train()

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Instanciez un agent de jeu humain non entrainable nommé <code style="text-align: left; font-size: 16px; color:#0f767a">human_player</code> </span></p>

In [92]:
human_player = StickPlayer(is_human=True, size=12, name="human_player", trainable=False)

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 4, choisissez l'action 3. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. </span></p>

In [93]:
# Choissiez de retirer 3 batonnets
state = 4

action = human_player.play(state)

# Calculate the value of the new game state
new_state = state - action

Entrez votre action (1, 2 ou 3): 3


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 4, choisissez l'action 5. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. </span></p>

In [57]:
# Choissiez de retirer 4 batonnets
state = 5
action = human_player.play(state)

# Calculate the value of the new game state
new_state = state - action

Entrez votre action (1, 2 ou 3): 5
Action invalide. Veuillez entrer 1, 2 ou 3.
Entrez votre action (1, 2 ou 3): 3


<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Dans l'état 1, choisissez l'action 3. Récupérez l'action choisie et calculez la valeur du nouvel état du jeu. </span></p>

In [94]:
# Choissiez de retirer 3 batonnets
state = 1
action = human_player.play(state)

# Calculate the value of the new game state
new_state = state - action


Entrez votre action (1, 2 ou 3): 3


<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🎮 <strong>4. Fonction de jeu</strong></span></h2>

La fonction de jeu permet de faire jouer deux joueurs au StickGame, d'enregistrer les transitions correspondant à chaque action des joueurs puis d'entrainer les deux agents si besoin. Les agents peuvent être aussi bien des agents artificiels que des agents humains

In [96]:
def play_game(game, p1, p2, train=True):

    '''
        Fonction qui permet de faire jouer deux agents (humains ou artificiels) au StickGame, d'enregistrer les
        transitions dans l'historique des agents, puis de faire l'entrainement des agents si besoin.

                Parameters:
                        - game (objet StickGame): objet instancié de la classe StickGame qui est un environnement de jeu
                        - p1 (objet StickPlayer) : objet instancié de la classe StickPlayer qui est un agent artificiel ou humain
                        - p2 (objet StickPlayer) : objet instancié de la classe StickPlayer qui est un agent artificiel ou humain
                        - train (boolééen) : Si l'argument train est activé (train=True), on entraine les deux agents


                Returns:
                        - Rien
    '''

    print("[INFO] : Starting new game !")

    # Commençons par ré-initialiser l'environnement de jeu pour s'assurer que l'environnement est dans son état initial
    state = game.reset()

    # Définissons la liste des deux joueurs
    # Le joueur en premier dans la liste est celui qui commence à jouer
    players = [p1, p2]

    # On utilise la fonction shuffle pour "mélanger" l'ordre des joueurs dans liste et donc choisir aléatoirement
    # le joueur qui commence à jouer en premier dans la partie
    random.shuffle(players)

    # Ici, p est le nombre de tours dans une partie.
    # On l'initialise à 0 avant le début d'une partie
    p = 0

    # On commence ici la boucle de jeu.
    # La boucle ne s'interrompt que lorsque le jeu est terminé, c'est-à-dire lorsqu'il n'y a plus de batonnets en jeu


<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>👩‍🏫 <strong>5. Entrainement de l'agent</strong></span></h2>

L'environnement, l'agent et la fonction de jeu sont maintenant conçus.
Nous pouvons maintenant passer à l'entrainement de l'agent afin qu'il puisse apprendre à exceller au StickGame. L'entrainement consiste à trouver la fonction de valeur optimale permettant à l'agent de trouver à chaque instant l'état qui a le plus de valeur.

In [97]:
N_EPISODES = 10000

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant d'entrainer un agent avec une stratégie "Duel d'agents" :
    <ul style="text-align: left; font-size: 16px; color:#0f767a">
    <li>Instancier un environnement de jeu StickGame</li>
    <li>Instancier deux agents IA </li>
    <li>Faites jouer les deux agents l'un contre l'autre pendant N_EPISODES parties</li>
    <li>Chaque 10 parties, diminuez le taux d'exploration de 0.4% sans jamais descendre en dessous de 0.05</li>
    <li>A la fin de chaque partie, entrainer les agents avec les transitions enregistrées</li>
  </ul> </span></p>

In [102]:
# Instancier un environnement de jeu StickGame
game = StickGame(12)

# Instancier deux agents IA
ai_player1 = StickPlayer(is_human=False, size=12, name="ai_player1", trainable=True)
ai_player2 = StickPlayer(is_human=False, size=12, name="ai_player2", trainable=True)

# Initialisation du taux d'exploration
eps = 0.99

# Boucle d'entrainement sur N_EPISODES parties
for episode in range(N_EPISODES):
    # Faire jouer les deux agents
    play_game(game, ai_player1, ai_player2)

    # Entrainer les agents
    ai_player1.train()
    ai_player2.train()

    # Diminuer le taux d'exploration tous les 10 épisodes
    if (episode + 1) % 10 == 0:
        eps = max(0.05, eps - 0.004)
        ai_player1.eps = eps
        ai_player2.eps = eps

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new g

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de calculer le taux de victoire de chacun des agents, trouvez le meilleur des deux agents puis ré-initialiser ces statistiques. </span></p>

In [116]:
# prompt: Ecrivez le code permettant de calculer le taux de victoire de chacun des agents, trouvez le meilleur des deux agents puis ré-initialiser ces statistiques

# Calculate win rates
n_games = 1000  # Number of games to play for evaluation
ai_player1_wins = 0
ai_player2_wins = 0

for _ in range(n_games):
    play_game(game, ai_player1, ai_player2, train=False)  # Don't train during evaluation
    if len(ai_player1.history) > len(ai_player2.history):  # Assuming more history means the player went last (won)
        ai_player1_wins +=1
    else:
        ai_player2_wins +=1

ai_player1_win_rate = ai_player1_wins / n_games
ai_player2_win_rate = ai_player2_wins / n_games

print(f"AI Player 1 Win Rate: {ai_player1_win_rate:.2%}")
print(f"AI Player 2 Win Rate: {ai_player2_win_rate:.2%}")

# Find the best agent
if ai_player1_win_rate > ai_player2_win_rate:
    best_agent = ai_player1
    print("AI Player 1 is the best agent.")
elif ai_player2_win_rate > ai_player1_win_rate:
    best_agent = ai_player2
    print("AI Player 2 is the best agent.")
else:
    print("Both agents have equal win rates.")


# Reset statistics
ai_player1.reset_stat()
ai_player2.reset_stat()
print("Agent statistics reset.")

[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Start

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quel est l'état de l'environnement le plus avantageux ? En d'autres termes, dans quel état vaut-il mieux se trouver pour avoir plus de chance de gagner ?</span></p>

In [117]:
best_states = []
best_value = -1

for state, value in best_agent.V.items():
  if value > best_value:
    best_states = [state]
    best_value = value
  elif value == best_value:
    best_states.append(state)

print(f"The most advantageous states are: {best_states} with value {best_value}")

The most advantageous states are: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] with value 0.0


<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Quel est l'état de l'environnement le moins avantageux ? En d'autres termes, dans quel état vaut-il mieux éviter de se trouver pour avoir plus de chance de gagner ?</span></p>

In [118]:

worst_states = []
worst_value = 1

for state, value in best_agent.V.items():
  if value < worst_value:
    worst_states = [state]
    worst_value = value
  elif value == worst_value:
    worst_states.append(state)

print(f"The least advantageous states are: {worst_states} with value {worst_value}")

The least advantageous states are: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] with value 0.0


<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Comparez les fonctions de valeurs des deux agents. Que remarquez-vous ?</span></p>

In [122]:
# Compare the value functions of the two agents.
print("Value function for ai_player1:")
print(ai_player1.V)
print("\nValue function for ai_player2:")
ai_player2.V

Value function for ai_player1:
{1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0}

Value function for ai_player2:


{1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0}

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  A partir de ces observations, quelle condition décide de la victoire de l'un ou l'autre des agents ?</span></p>

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🧪 <strong>6. Test de l'agent entrainé contre un agent "random"</strong></span></h2>

Dans cette partie, nous faisons jouer notre meilleur agent entrainé contre un agent artificiel "random", c'est-à-dire un agent qui ne prend que des décisions aléatoires.

In [124]:
# Instancier un agent random
random_player = StickPlayer(is_human=False, size=12, name="random_player", trainable=False)
random_player.play = lambda state: randint(1, min(state, 3))

# Faire jouer l'agent random contre le meilleur agent pendant N_EPISODES parties
N_EPISODES = 100
random_wins = 0
best_agent_wins = 0

for _ in range(N_EPISODES):
    play_game(game, best_agent, random_player, train=False)
    if len(best_agent.history) > len(random_player.history):
        best_agent_wins += 1
    else:
        random_wins += 1

print(f"Best Agent Wins: {best_agent_wins}")
print(f"Random Agent Wins: {random_wins}")

[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Start

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de calculer le taux de victoire de chacun des deux agents
    </span></p>

In [125]:

# Calculate win rates
n_games = 1000  # Number of games to play for evaluation
ai_player1_wins = 0
ai_player2_wins = 0

for _ in range(n_games):
    play_game(game, ai_player1, ai_player2, train=False)
    if len(ai_player1.history) > len(ai_player2.history):
        ai_player1_wins += 1
    else:
        ai_player2_wins += 1

ai_player1_win_rate = ai_player1_wins / n_games
ai_player2_win_rate = ai_player2_wins / n_games

print(f"AI Player 1 Win Rate: {ai_player1_win_rate:.2%}")
print(f"AI Player 2 Win Rate: {ai_player2_win_rate:.2%}")

[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Starting new game !
[INFO] : Start

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓  Il est sureprenant de voir l'agent random gagner tout de même quelques parties. A quoi cela peut-il être dû ?</span></p>

<h2 style="text-align: left; color:#0099cc;font-size: 25px"><span>🧪 <strong>6. Test de l'agent entrainé contre un agent humain</strong></span></h2>

<p style="text-align: left; font-size: 16px; color:#0f767a"><span>🖥️  Ecrivez le code permettant de faire jouer le meileur agent IA contre un agent humain, c'est-à-dire vous:
    <ul style="text-align: left; font-size: 16px; color:#0f767a">
    <li>Instancier un agent humain</li>
    <li>Jouer en boucle contre l'agent IA entrainé</li>
  </ul>
    </span></p>

In [128]:
# Instancier un agent humain
human_player = StickPlayer(is_human=True, size=12, name="human_player", trainable=False)

# Jouer en boucle contre l'agent IA entrainé
while True:
    play_game(game, best_agent, human_player, train=False)

    another_game = input("Do you want to play another game? (yes/no): ")
    if another_game.lower() != "yes":
        break

[INFO] : Starting new game !
Do you want to play another game? (yes/no): no
